# Notebook for computing hashes, buckets and similarity values for the grid scheme. 

Utilizes the grid scheme

Incorporates:
* Hashing of trajectories using grid scheme
* Bucketing of hashes made from grid scheme
* Similarity computation between trajectories within buckets.
    * Both for DTW and Frechet

Produces:
* JSON file containing buckets
* Similarity values for trajectories within buckets


In [3]:

# Importing nescessary modules
import os
import sys
import shutil
import timeit as ti
from tqdm import tqdm

from multiprocessing import Pool

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

from utils.helpers.save_trajectory import save_trajectory_hashes
from utils.helpers import file_handler as fh
from utils.helpers import metafile_handler as mfh
from schemes.lsh_disk import DiskLSH
import json
from computation.disk_similarity import generate_disk_hash_similarity
from disk_similarity import generate_disk_hash_similarity

import os

Project root found: c:\Users\eivin\dev\JoonEndreLSH\masteroppgave


ModuleNotFoundError: No module named 'disk_similarity'

# Rome


### DTW


In [2]:
measure = "dtw"
similarities = generate_disk_hash_similarity(
    city="rome", diameter=0.6, layers=5, disks=50, measure=measure, size=50
)
output_path = f"similarity_values/disk/rome/disk_rome-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))

### Frechet


In [ ]:
measure = "frechet"
similarities = generate_disk_hash_similarity(
    city="rome", diameter=1.6, layers=5, disks=100, measure=measure, size=500
)
output_path = f"similarity_values/disk/rome/disk_rome-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))

# Porto


In [ ]:
measure = "dtw"
similarities = generate_disk_hash_similarity(
    "porto", diameter=2.2, layers=4, disks=60, measure=measure, size=50
)
output_path = f"similarity_values/disk/porto/disk_porto-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))